# Chapter 10 Data Aggregation and Group Operations

- Split a data frame into pieces using one or more keys.
- Calculate group summary statistics such as count, mean, standard deviation, or a user-defined function.
- Apply within-group transformations such as normalization.
- Compute pivot tables and cross-tabulations.
- Perform statistical group analysis.

## I. GroupBy Mechanics

Many data processing follows a **split-apply-combine** process. For example, you may want to do the following operations to analyze a dataset about sales:
1. What is the total revenue every day?
2. What is the total sales of each product?
3. How much has each client perchased in total?

These operations all requires that you split the data into groups, and then apply certain calculations to each of the groups, and finally combine all results into a new table. In Pandas this is mostly done with `groupby()` function.

In [1]:
import numpy as np
import pandas as pd

In [2]:
# An example:
df = pd.DataFrame({'key1' : ['a', 'a', 'b', 'b', 'a'],
                   'key2' : ['one', 'two', 'one', 'two', 'one'],
                   'data1' : np.random.randn(5),
                   'data2' : np.random.randn(5)})
df

,key1,key2,data1,data2
0,a,one,1.533264,0.788511
1,a,two,0.070856,-0.193073
2,b,one,0.598159,1.251546
3,b,two,-0.671643,-0.619285
4,a,one,0.331091,0.642477


In [3]:
df['key1']

0    a
1    a
2    b
3    b
4    a
Name: key1, dtype: object

In [4]:
df['data1']

0    1.533264
1    0.070856
2    0.598159
3   -0.671643
4    0.331091
Name: data1, dtype: float64

In [5]:
# Split data1 values according to key1:
groups = df['data1'].groupby(df['key1'])
groups

In [6]:
# Apply mean() function to find the average value for each group
means = groups.mean()
means

key1
a    0.645070
b   -0.036742
Name: data1, dtype: float64

In [7]:
# Convert it to a data frame
df_means = means.to_frame(name='data1_mean')
df_means

,data1_mean
key1,
a,0.645070
b,-0.036742


In [8]:
# Put all operations in one statement
df_means = df['data1'].groupby(df['key1']).mean().to_frame(name='data1_mean')
df_means

,data1_mean
key1,
a,0.645070
b,-0.036742


In [9]:
# Exercise: split data2 according to key2, and calculate the sum.

# 1. split
groups2 = df['data2'].groupby(df['key2'])
# 2. apply
results = groups2.sum()
# 3. convert the result to a data frame
results.to_frame(name="data2_sum")

,data2_sum
key2,
one,2.682535
two,-0.812358


We can use more than one column as keys.

In [10]:
# Split the data according to both key1 and key2
groups = df['data1'].groupby([df['key1'], df['key2']])

In [11]:
# Calculate the mean
means = groups.mean()
means

key1  key2
a     one     0.932177
      two     0.070856
b     one     0.598159
      two    -0.671643
Name: data1, dtype: float64

In [12]:
means.to_frame(name="Value")

Value
key1 key2          
a    one   0.932177
     two   0.070856
b    one   0.598159
     two  -0.671643

We obtain a pandas Series with **hierarchical indexing**. It can be converted to a data frame using `unstack()`.

In [13]:
# Convert it to a data frame
means.unstack()

key2,one,two
key1,,
a,0.932177,0.070856
b,0.598159,-0.671643


In [14]:
means.unstack(level=0)

key1,a,b
key2,,
one,0.932177,0.598159
two,0.070856,-0.671643


In [15]:
# Put all operations in one statement

# df['data1'].groupby([df['key1'], df['key2']]).mean()
df['data1'].groupby([df['key1'], df['key2']]).mean().to_frame(name="Value").unstack()

Value          
key2       one       two
key1                    
a     0.932177  0.070856
b     0.598159 -0.671643

In [16]:
# Split the entire data frame
df.groupby([df['key1'], df['key2']]).mean()

data1     data2
key1 key2                    
a    one   0.932177  0.715494
     two   0.070856 -0.193073
b    one   0.598159  1.251546
     two  -0.671643 -0.619285

In [17]:
# Frequently the grouping information is found in the same data frame as the data 
# you want to work on. In that case, simply put column names as the keys:
df.groupby(['key1', 'key2']).mean()

data1     data2
key1 key2                    
a    one   0.932177  0.715494
     two   0.070856 -0.193073
b    one   0.598159  1.251546
     two  -0.671643 -0.619285

In [18]:
# Find the number of instances in each subgroup
df.groupby(['key1', 'key2']).size().to_frame(name='size')

size
key1 key2      
a    one      2
     two      1
b    one      1
     two      1

**Iterating Over Groups**

The GroupBy object support iteration, providing a sequence of 2-tuples containing the group name along with the data.

In [19]:
# Show the content of each group.
groups = df.groupby(['key1', 'key2'])
for name, group in groups: # groups = [(("a", "one"), ....),
#                                       ("a", "two"), ....), ....]
    print("Name:", name)
    print(group)

Name: ('a', 'one')
  key1 key2     data1     data2
0    a  one  1.533264  0.788511
4    a  one  0.331091  0.642477
Name: ('a', 'two')
  key1 key2     data1     data2
1    a  two  0.070856 -0.193073
Name: ('b', 'one')
  key1 key2     data1     data2
2    b  one  0.598159  1.251546
Name: ('b', 'two')
  key1 key2     data1     data2
3    b  two -0.671643 -0.619285


**Syntactic sugar**: selecting columns for groupby()

In [20]:
df.groupby('key1')['data1'].min()

key1
a    0.070856
b   -0.671643
Name: data1, dtype: float64

In [21]:
df['data1'].groupby(df['key1']).min()

key1
a    0.070856
b   -0.671643
Name: data1, dtype: float64

In [22]:
# The following statement does not work because 'key1' is not a column in df['data1']
df['data1'].groupby('key1').min()

KeyError: 'key1'

In [ ]:
df.groupby('key1')[['data2']].min()

In [ ]:
df[['data2']].groupby(df['key1']).min()

**Grouping with dictionary**

In [ ]:
values = np.array([
    [100, 80, 95],
    [55, 60, 45],
    [70, 75, 90],
    [75, 70, 60],
    [60, 73, 75],
    [72, 63, 70]
])
data = pd.DataFrame(values,
                   columns=['Midterm', 'Project', 'Final'],
                   index=['Alice', 'Bob', 'Chris', 'Doug', 'Eva', "Frank"])
data

In [ ]:
gender = {
    'Alice': 'F',
    'Bob': 'M',
    'Chris': 'M',
    'Doug': 'M',
    'Eva': 'F',
    'Frank': 'M'
}

In [ ]:
# split the rows according to gender
data.groupby(gender).size()

In [ ]:
data.groupby(['F', 'M', 'M', 'M', 'F', 'M']).size() # not recommended

In [ ]:
data.groupby(gender).mean()

**Grouping with functions**

Any function passed as a group key will be called once per index value, with the returned values being used as the group names.

In [ ]:
def get_initial(name):
    return name[0]

In [ ]:
data.groupby(get_initial).mean()

In [ ]:
data.groupby(lambda x: x[0]).mean()

In [ ]:
data.groupby(len).mean()

In [ ]:
len("Alice")

In [ ]:
len("Bob")

**Example: Filling Missing Values with Group-Specific Values**

In [ ]:
states = ['Ohio', 'New York', 'Vermont', 'Florida',
          'Oregon', 'Nevada', 'California', 'Idaho']
group_key = ['East'] * 4 + ['West'] * 4
data = pd.DataFrame(np.random.randn(8), index=states, columns=['Value'])
data.loc[['Vermont', 'Nevada', 'Idaho']] = np.nan
data['group_key'] = group_key
data

In [ ]:
# Fill the missing values with mean value

data.fillna(data.mean())

In [ ]:
# Find the average value of eastern states and western states
means = data.groupby("group_key").mean()

# Fill missing values with group specific average
# data.groupby("group_key").apply(lambda x: x.fillna(x.mean()))
def fill_group(group):
    return group.fillna(group.mean())
data.groupby("group_key").apply(fill_group)

In [ ]:
# Fill missing values with the following rule:
# East: 0.5
# West: -0.5
values = {'East': 0.5,
          'West': -0.5}
# data.groupby("group_key").apply(lambda x: x.fillna(values[x.name]))
def fill_group2(group):
    value = values[group.name]
    return group.fillna(value)
data.groupby("group_key").apply(fill_group2)

**Example: Random Sampling and Permutation**

In [ ]:
# Hearts, Spades, Clubs, Diamonds
suits = ['H', 'S', 'C', 'D']
card_val = (list(range(1, 11)) + [10] * 3) * 4
base_names = ['A'] + list(range(2, 11)) + ['J', 'Q', 'K']
cards = []
for suit in ['H', 'S', 'C', 'D']: # heart, spade, club, diamond
    cards.extend(str(num) + suit for num in base_names)

deck = pd.Series(card_val, index=cards)
deck

In [ ]:
# Randomly sample 5 rows

deck.sample(5)

In [ ]:
# Randomly sample 2 cards from each suit
groups = deck.groupby(lambda x: x[-1])
# for name, group in groups:
#     print(name)
#     print(group)
groups.apply(lambda x: x.sample(2))

**Example: Analyzing Cell Phone History**

In [ ]:
# Load data
# https://www.shanelynn.ie/summarising-aggregation-and-grouping-data-in-python-pandas/
url = "https://shanelynnwebsite-mid9n9g1q9y8tt.netdna-ssl.com/wp-content/uploads/2015/06/phone_data.csv"
# data = pd.read_csv(url, delimiter=",")
data = pd.read_csv(url, delimiter=",", index_col='index')
print(data.shape)
data.head(20)

1. **date**: The date and time of the entry
2. **duration**: The duration (in seconds) for each call, the amount of data (in MB) for each data entry, and the number of texts sent (usually 1) for each sms entry.
3. **item**: A description of the event occurring – can be one of call, sms, or data.
4. **month**: The billing month that each entry belongs to – of form ‘YYYY-MM’.
5. **network**: The mobile network that was called/texted for each entry.
6. **network_type**: Whether the number being called was a mobile, international (‘world’), voicemail, landline, or other (‘special’) number.

In [ ]:
data.dtypes

In [ ]:
# Convert date column from string to datetime objects
from dateutil.parser import parse
data['date'] = data['date'].apply(parse, dayfirst=True)
data.head(3)

In [ ]:
# Check data types

data.dtypes

In [ ]:
# Check missing values

data.isnull().sum()

**Apply GroupBy actions**

In [ ]:
# Which months are covered in this data set?
# data['month'].unique()
# set(data['month'])

data.groupby(['month']).groups.keys()

In [ ]:
# Find the first entry for each month
data.groupby(['month']).first()

# without using groupby
# months = data['month'].unique()
# result = pd.DataFrame(columns=data.columns)
# for month in months:
#     subdata = data[data['month'] == month]
#     instance = subdata.loc[[subdata.index[0]], :]
# #     print(instance)
#     result = pd.concat([result, instance])
# result  

In [ ]:
# Get the number of instances in each month

# data.groupby('month').size()
data.groupby('month')['date'].count()

In [ ]:
# What is the sum of call durations for each month?

# data[data['item'] == 'call'].groupby('month').sum()
data.groupby(['month', 'item']).sum().unstack()

**Group by more than one variable**

In [ ]:
# How many calls, messages, and data entries are there in each month?

# data.groupby(['month', 'item'])['duration'].count()

data.groupby(['month', 'item'])['duration'].count().to_frame(name='frequency')\
    .unstack()

In [ ]:
# How many instances are there per month, split by network_type?

# data.groupby(['network_type', 'month']).size()
data.groupby(['network_type', 'month']).size().to_frame("Frequency")\
.unstack(level=0)

## II. Data Aggregation
Aggregation refer to any data transformation that produces numeric values from arrays. The preceding examples have used several of them, including `mean()`, `count()`, `first()`, `min()`, and `sum()`. However, user-defined functions can also be applied to create desired summary.

In [ ]:
# Define function range() that returns(max - min)
def get_range(array):
    return array.max() - array.min()

In [ ]:
# Apply agg() to find the range of each type of cell phone use.
data.groupby(['item'])['duration'].agg(get_range)

In [ ]:
subdata1 = data[data['item'] == 'data']
subdata1.head()

In [ ]:
subdata1['duration'].value_counts()

In [ ]:
# If only one function is applied, there is no difference in agg() and apply()
data.groupby(['item'])['duration'].apply(get_range)

In [ ]:
# Apply multiple aggregation functions
data.groupby(['item'])['duration'].agg([get_range, np.max, np.min])

In [ ]:
# Declare columns names
data.groupby(['item'])['duration'].agg([('range', get_range),
                                        ('maximum', np.max),
                                        ('minimum', np.min)])

In [ ]:
# Apply a different function to each column
functions = {
    'duration': sum,
    'network_type': 'count',
    'date': 'first'
}
data.groupby(['month', 'item']).agg(functions)

In [ ]:
# Tuple named aggregations
data[data['item'] == 'call'].groupby('month').agg(
    # Get max of the duration column for each group
    max_duration=('duration', max),
    # Get min of the duration column for each group
    min_duration=('duration', min),
    # Get sum of the duration column for each group
    total_duration=('duration', sum),
    # Apply a lambda to date column
    num_days=("date", lambda x: (max(x) - min(x)).days)   
)

## III. Pivot Table
It is used to split the data using two sets of keys.

In [ ]:
# Create a pivot table with counts for each month and network type
data.pivot_table('date', index='month', columns='network_type', aggfunc=len)

## IV. Cross Tabulation

In [ ]:
pd.crosstab(index=data['month'], columns=data['network_type'])

# Homework:
Use the cell phone usage data in this exercise.
1. Find out the network names that belongs to network_type "mobile".
2. How many messages were sent to each mobile network every month?
3. What is the total call duration to each mobile network every month?